### 머신러닝 지도학습: 분류_tree

In [ ]:
# 1> 결정트리 DecisionTreeClassifier: 와인 데이터 실습
from sklearn.datasets import load_wine
import pandas as pd
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df['class'] = wine.target
print(wine.target_names)
df.head()

In [ ]:
# 학습 데이터 준비
from sklearn.model_selection import train_test_split
X_data = df.iloc[:, :-1]
y_data = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, 
                                test_size=0.2, random_state= 156)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
# 1_보충> 결정트리 DecisionTreeClassifier:

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
X_data = wine.data
y_data = wine.target
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state= 156)
dt_clf = DecisionTreeClassifier(random_state=156)
dt_clf.fit(X_train , y_train)
pred = dt_clf.predict(X_test)
accuracy = accuracy_score(y_test , pred)
print('결정 트리 예측 정확도: {0:.4f}'.format(accuracy))

In [ ]:
# 1-1. GridSearchCV 하이퍼 파라미터 찾기

from sklearn.model_selection import GridSearchCV
parameters = {'criterion':['gini', 'entropy'],'max_depth':[None, 2, 3, 5, 7], 
                'min_samples_split':[2,3,5,7],'min_samples_leaf':[1,3,5,7]}
grid_dt = GridSearchCV(dt_clf, param_grid=parameters, cv=5)
grid_dt.fit(X_train, y_train)
scores_df = pd.DataFrame(grid_dt.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score']]
model = grid_dt.best_estimator_
pred = model.predict(X_test)
accuracy_score(y_test, pred)
print('GridSearchCV 최적 파라미터 :', grid_dt.best_params_)
print(f'GridSearchCV 최고 정확도 : {grid_dt.best_score_:.4f}')

In [ ]:
# 1-2 피처 중요도 그래프 .feature_importances_
import seaborn as sns
import matplotlib.pyplot as plt
ftr_importances_values = model.feature_importances_
ftr_importances = pd.Series(ftr_importances_values,index=wine.feature_names)
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]
plt.figure(figsize=(8,6))
plt.title('Feature importances Top 20')
sns.barplot(x=ftr_top20 , y = ftr_top20.index)

In [ ]:
# 1-3 트리 구조 그래프
from sklearn.tree import plot_tree
plt.figure(figsize=(10,7), dpi=1200)
plot_tree(model, filled=True, feature_names=wine.feature_names, 
    class_names=list(wine.target_names))

In [ ]:
# 과적합 확인하는 방법
# cross_val_score: 학습데이터 평가점수 vs 테스트데이터 평가점수 --> 비슷해야함
# max_dept 적은 수/ min_samples_split: split 하는 최소 샘플 수/min_samples_leaf: 리프노드에 있을 최소 샘플갯수
    
from sklearn.model_selection import cross_val_score
dt_clf=DecisionTreeClassifier(random_state=0)
dt_clf.fit(X_train, y_train)
scores=cross_val_score(dt_clf, X_train, y_train, scoring='accuracy', cv=5)
print('교차검증 평균 평가',scores.mean())
print('테스트데이터 평가점수',accuracy_score(y_test, dt_clf.predict(X_test)))

In [ ]:
#트리 구조 그래프
from sklearn.tree import plot_tree
plt.figure(figsize=(10,7), dpi=1200)
plot_tree(model, filled=True, feature_names=wine.feature_names, 
        class_names=list(wine.target_names))

#### 머신러닝 분류- 앙상블 학습 ensemble

In [ ]:
# VotingClassifier 보팅분류기
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
cancer = load_breast_cancer()
data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head(3)

In [ ]:
lr_clf = LogisticRegression(solver='liblinear')
dt_clf = DecisionTreeClassifier(random_state=0)
vo_clf = VotingClassifier( estimators=[('LR',lr_clf), ('DT',dt_clf)] , voting='soft' )
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2 , random_state= 0 )
vo_clf.fit(X_train , y_train)
pred = vo_clf.predict(X_test)
print(f'Voting 분류기 정확도: {accuracy_score(y_test,pred):.4f}')

In [ ]:
# 각 모델의 정확도 계산
classifiers = [lr_clf, dt_clf]
for clf in classifiers:
    clf.fit(X_train , y_train)
    pred = clf.predict(X_test)
    class_name= clf.__class__.__name__
    print(f'{class_name} 정확도: {accuracy_score(y_test, pred):.4f}')

In [ ]:
# 랜덤포레스트 RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                    test_size=0.2 , random_state= 0)
rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test , pred)
print(f'랜덤 포레스트 정확도: {accuracy:.4f}')

In [ ]:
# 랜덤포레스트 파이퍼 파라미터 튜닝 GridSearchCV
from sklearn.model_selection import GridSearchCV
params = {'n_estimators':[100], 'max_depth' : [6, 8, 10, 12], 
            'min_samples_leaf' : [8, 12, 18 ],'min_samples_split' : [8, 16, 20]}
rf_clf = RandomForestClassifier(random_state=0)
grid_cv = GridSearchCV(rf_clf , param_grid=params , cv=3, n_jobs=-1, verbose=True)
grid_cv.fit(X_train , y_train)

print('GridSearchCV 최적 파라미터:', grid_cv.best_params_)
print(f'GridSearchCV 최고 정확도: {grid_cv.best_score_:.4f}')

In [ ]:
model = grid_cv.best_estimator_
pred = model.predict(X_test)
accuracy_score(y_test, pred)

In [ ]:
# 피쳐 중요도 그래프
import matplotlib.pyplot as plt
import seaborn as sns
ftr_importances_values = model.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index=cancer.feature_names )
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]
plt.figure(figsize=(8,6))
plt.title('Feature importances Top 20')
sns.barplot(x=ftr_top20 , y = ftr_top20.index)
plt.show()
plt.draw()

In [ ]:
# GBM GradientBoostingClassifier

from sklearn.ensemble import GradientBoostingClassifier
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                    test_size=0.2 , random_state= 0)
gb_clf = GradientBoostingClassifier(random_state=0, verbose=True)
gb_clf.fit(X_train , y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print(f'GBM 정확도: {gb_accuracy:.4f}')

####  XGBoost

In [ ]:
#  xgboost XGBoodt
import xgboost as xgb
from xgboost import plot_importance
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset = load_breast_cancer()
cancer_df = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
cancer_df['target'] = dataset.target
X_features = cancer_df.iloc[:, :-1]
y_label = cancer_df.iloc[:, -1]
X_train, X_test, y_train, y_test=train_test_split(X_features, y_label, 
                test_size=0.2, random_state=156 )
X_tr, X_val, y_tr, y_val= train_test_split(X_train, y_train,
                test_size=0.1, random_state=156 )

In [ ]:
dtr = xgb.DMatrix(data=X_tr, label=y_tr)
dval = xgb.DMatrix(data=X_val, label=y_val)
dtest = xgb.DMatrix(data=X_test , label=y_test)
params = {'max_depth':3,'eta': 0.05,'objective':'binary:logistic','eval_metric':'logloss'}
num_rounds = 400

eval_list = [(dtr,'train'),(dval,'eval')] 
# 또는 eval_list = [(dval,'eval')]
xgb_model = xgb.train(params = params , dtrain=dtr, num_boost_round=num_rounds , 
            early_stopping_rounds=50, evals=eval_list )

In [ ]:
pred_probs = xgb_model.predict(dtest)
print('predict( ) 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨')
print(np.round(pred_probs[:10],3))
preds = [ 1 if x > 0.5 else 0 for x in pred_probs ]
print('예측값 10개만 표시:',preds[:10])

In [ ]:
# 그래프 그리기
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 7))
plot_importance(xgb_model, ax=ax)

In [ ]:
#래퍼 XGBClassifier 개요 및 적용
from xgboost import XGBClassifier 
xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
evals = [(X_test, y_test)]
xgb_wrapper.fit(X_train, y_train, early_stopping_rounds = 100, 
eval_metric = "logloss", eval_set=evals)
preds = xgb_wrapper.predict(X_test)
pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]

In [ ]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print(f'정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}')
    print(f'F1: {f1:.4f}, AUC:{roc_auc:.4f}')

In [ ]:
get_clf_eval(y_test, ws100_preds, ws100_pred_proba)

In [ ]:
#스태킹 앙상블

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
# 기본 스태킹 모델
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
cancer_data = load_breast_cancer()
X_data = cancer_data.data
y_label = cancer_data.target
X_train , X_test , y_train , y_test = train_test_split(X_data, y_label, 
test_size=0.2 , random_state=0)

In [ ]:
# 개별 ML 모델을 위한 Classifier 생성.
knn_clf =KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier(random_state=0)
ada_clf = AdaBoostClassifier(n_estimators=100)
# 최종 Stacking 모델을 위한 Classifier생성. 
lr_final = LogisticRegression(solver='liblinear')

In [ ]:
# 개별 모델들을 학습. 
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train , y_train)
dt_clf.fit(X_train , y_train)
ada_clf.fit(X_train, y_train)
# 학습된 개별 모델들이 각자 반환하는 학습 데이터 셋을 생성
knn_train = knn_clf.predict(X_train)
rf_train = rf_clf.predict(X_train)
dt_train = dt_clf.predict(X_train)
ada_train = ada_clf.predict(X_train)

In [ ]:
# 학습된 개별 모델들이 각자 반환하는 테스트 데이터 셋을 생성
knn_test = knn_clf.predict(X_test)
rf_test = rf_clf.predict(X_test)
dt_test = dt_clf.predict(X_test)
ada_test = ada_clf.predict(X_test)
# 학습 데이터와 테스트 데이터로 합치기
pred_train = np.vstack([knn_train, rf_train, dt_train, ada_train]).T
pred_test = np.vstack([knn_test, rf_test, dt_test, ada_test]).T

In [ ]:
lr_final.fit(pred_train, y_train)
final = lr_final.predict(pred_test)
print(f'최종 모델의 정확도: {accuracy_score(y_test , final):.4f}')

In [ ]:
# 스태킹 실습
from sklearn.ensemble import StackingClassifier
stack = StackingClassifier([('knn', knn_clf),('rf', rf_clf),('dt', dt_clf),
                ('ada', ada_clf)], final_estimator = lr_clf)
stack.fit(X_train, y_train)
pred = stack.predict(X_test)
accuracy_score(y_test, pred)

#### ensemble 학습
1) 보팅 Voting  
        :한 데이터셋 + 분류기 다수:여러가지 알고리즘 사용, 다수결 원칙으로 투표로 결과 예측
2) 배깅 Bagging  
        : 하나의 알고리즘을 다양한 데이터로 학습하여 예측결과를 통합/ RandomForest
3) 부스팅 Boosting : 순차적으로 학습-예측하면서 오류개선하며 가중치 조정.딥러닝 학습방식 
        부스팅: 하나의 알고리즘 여러번 순차적으로 학습시켜서 가중치를 업데이트하는 방법
     : GBM, AdaBoost, XGBoost, LightGBM(속도 좋고, 과적합 문제 약간), 
        CatBoost(속도 좋고, 과적합 개선)
4) 스태킹 Stacking : 성능이 비슷한 약한 여러 모델로 작업하여 예측한 값을 최종 모델을 사용

In [ ]:
# 1) ensemble 학습: Voting

from sklearn.ensemble import VotingClassifier
lr_clf = LogisticRegression(solver='liblinear')
dt_clf = DecisionTreeClassifier(random_state=0)
knn_clf= KNeighborsClassifier()
vo_clf = VotingClassifier( estimators=[('LR',lr_clf), ('DT',dt_clf),('KNN', knn_clf)] , voting='soft' )

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2 , random_state= 0)
vo_clf.fit(X_train , y_train)
pred = vo_clf.predict(X_test)
print(f'Voting 분류기 정확도: {accuracy_score(y_test,pred):.4f}')

classifiers = [lr_clf, dt_clf, knn_clf]
for clf in classifiers:
    clf.fit(X_train , y_train)
    pred = clf.predict(X_test)
    class_name= clf.__class__.__name__
    print(f'{class_name} 정확도: {accuracy_score(y_test, pred):.4f}')

In [ ]:
# 2) ensemble 학습: RandomForestClassifier

from sklearn.ensemble import RandomForestClassifier
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2 , random_state= 0)
rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test , pred)
print(f'랜덤 포레스트 정확도: {accuracy:.4f}')

# 랜덤포레스트 파이퍼 파라미터 튜닝 GridSearchCV
params = {'n_estimators':[100], 'max_depth' : [6, 8, 10, 12], 
          'min_samples_leaf' : [8, 12, 18 ],'min_samples_split' : [8, 16, 20]}
rf_clf = RandomForestClassifier(random_state=0)
grid_cv = GridSearchCV(rf_clf , param_grid=params , cv=3, n_jobs=-1, verbose=True)
grid_cv.fit(X_train , y_train)
print('GridSearchCV 최적 파라미터:', grid_cv.best_params_)
print(f'GridSearchCV 최고 정확도: {grid_cv.best_score_:.4f}')
model = grid_cv.best_estimator_
pred = model.predict(X_test)
accuracy_score(y_test, pred)

# 피쳐 중요도 그래프
ftr_importances_values = rf_clf.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index=cancer.feature_names )
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]
plt.figure(figsize=(8,6))
plt.title('Feature importances Top 20')
sns.barplot(x=ftr_top20 , y = ftr_top20.index)

In [ ]:
# 3) ensemble 학습: GBM/ GradientBoostingClassifier

from sklearn.ensemble import GradientBoostingClassifier
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2 , random_state= 0)
gb_clf = GradientBoostingClassifier( n_estimators=200,learning_rate=0.05,, random_state=0,verbose=True )
gb_clf.fit(X_train , y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print(f'GBM 정확도: {gb_accuracy:.4f}')

In [ ]:
# 4) ensemble 학습: 사이킷런 XGBClassifier 

from xgboost import XGBClassifier 
dataset = load_breast_cancer()
cancer_df = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
cancer_df['target'] = dataset.target
X_features = cancer_df.iloc[:, :-1]
y_label = cancer_df.iloc[:, -1]
X_train, X_test, y_train, y_test=train_test_split(X_features, y_label,test_size=0.2, random_state=156 )
X_tr, X_val, y_tr, y_val= train_test_split(X_train, y_train,test_size=0.1, random_state=156 )
xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
evals = [(X_val, y_val)]
xgb_wrapper.fit(X_tr, y_tr, eval_metric = "logloss",early_stopping_rounds=10,eval_set=evals)
preds = xgb_wrapper.predict(X_test)
pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]
accuracy_score(y_test, pred)

# xgb_wrapper.feature_importances_ 로 그래프
ftr_importances_values = xgb_wrapper.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index=cancer.feature_names )
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]
plt.figure(figsize=(8,6))
plt.title('Feature importances Top 20')
sns.barplot(x=ftr_top20 , y = ftr_top20.index)

In [ ]:
# 5) 스태킹 앙상블
# 4개의 모델(KNN, RF, Ada, DT)를 통해 개별 예측/최종 Logistic 모델을 사용하여 예측 예정

In [ ]:
from sklearn.ensemble import StackingClassifier

from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

stack = StackingClassifier([('Gaus', GaussianNB()),
                            ('DT', DecisionTreeClassifier(random_state=0)),
                            ('Knn', KNeighborsClassifier())],
                           final_estimator=LogisticRegression(random_state=0),stack_method='predict')
stack.fit(X_train, y_train)
pred = stack.predict(X_test)
pred_proba = stack.predict_proba(X_test)

get_clf_eval(y_test, pred, pred_proba)

### 분류 - Tree 외

In [ ]:
# 1. 로지스틱 회귀: 시그모이드함수를 생성하고 입력값이 0인지 1인지 예측하여 수랭
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
pred_proba = lr_clf.predict_proba(X_test)[:,1]
acc = accuracy_score(y_test, pred)
auc = roc_auc_score(y_test, pred_proba)
print(f'accuracy: {acc:.3f}, roc_auc:{auc:.3f}')

# 최적의 파라미터
from sklearn.model_selection import GridSearchCV
params={'solver':['liblinear', 'lbfgs'],'penalty':['l2', 'l1'],'C':[0.01, 0.1, 1, 5, 10]}
lr_clf = LogisticRegression()
grid_clf = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=3 )
grid_clf.fit(cancer.data, cancer.target)
print(f'최적 하이퍼 파라미터:{grid_clf.best_params_}') 
print(f'최대 평균 정확도:{grid_clf.best_score_:.3f}')

# GridSearchCV 결과보기
pd.DataFrame(grid_clf.cv_results_)[['mean_test_score', 'rank_test_score']] # 필요한 파라미터로
lr_clf.coef_, lr_clf.intercept_   # 주의! 되나 봐야함

In [ ]:
# 2. 최근접 이웃
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7, metric='euclidean')
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
pred_proba = knn.predict_proba(X_test)[:,1]
acc = accuracy_score(y_test, pred)
auc = roc_auc_score(y_test, pred_proba)
print(f'accuracy: {acc:.3f}, roc_auc:{auc:.3f}')

In [ ]:
# 3. 서포트 벡터 머신(KVM)
from sklearn.svm import SVC, SVR
svc = SVC(probability = True)   # pred_proba 구할 경우 True
svc.fit(X_train, y_train)
pred = svc.predict(X_test)
pred_proba = svc.predict_proba(X_test)[:,1]
acc = accuracy_score(y_test, pred)
auc = roc_auc_score(y_test, pred_proba)
print(f'accuracy: {acc:.3f}, roc_auc:{auc:.3f}')

# 3-1. kernel 비교하기
kernels = ['rbf', 'linear', 'poly', 'sigmoid']
for kernel in kernels:
    print(kernel)
    svc = SVC(kernel=kernel, probability = True)
    svc.fit(X_train, y_train)
    pred = svc.predict(X_test)
    pred_proba = svc.predict_proba(X_test)[:,1]
    acc = accuracy_score(y_test, pred)
    auc = roc_auc_score(y_test, pred_proba)
    print(f'accuracy: {acc:.3f}, roc_auc:{auc:.3f}')

# 3-2. 파라미터 찾기 GridSearchCV
params = {'C' : [0.01, 0.1, 1, 2, 5],'gamma' : [0.01, 0.05,0.1, 0.5, 1, 2, 5]}
grid_cv = GridSearchCV(svc, param_grid=params, scoring='accuracy', cv = 3, verbose=True)
grid_cv.fit(X_train, y_train)
